<a href="https://colab.research.google.com/github/IshaPatro/Investopedia-Knowledge-Graph/blob/main/Scrape_Investopedia_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import sys
!{sys.executable} -m pip install pip install python-slugify
!{sys.executable} -m pip install pip install bs4

In [22]:
import requests, random, logging, urllib.request, json
from bs4 import BeautifulSoup
from tqdm import tqdm
from slugify import slugify
import requests
import pandas as pd
import concurrent.futures
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import os
import csv
import time

In [23]:
url = 'https://www.investopedia.com/financial-term-dictionary-4769738'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    print("Successfully fetched the page")
else:
    print(f"Failed to fetch the page, status code: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')
url_elements = soup.find_all('a', class_='dictionary-top24-list__complete-listing-link')
urls = [el['href'] for el in url_elements]

for url in urls:
    print(url)

Successfully fetched the page
https://www.investopedia.com/terms-beginning-with-num-4769350
https://www.investopedia.com/terms-beginning-with-a-4769351
https://www.investopedia.com/terms-beginning-with-b-4769352
https://www.investopedia.com/terms-beginning-with-c-4769353
https://www.investopedia.com/terms-beginning-with-d-4769354
https://www.investopedia.com/terms-beginning-with-e-4769355
https://www.investopedia.com/terms-beginning-with-f-4769356
https://www.investopedia.com/terms-beginning-with-g-4769357
https://www.investopedia.com/terms-beginning-with-h-4769358
https://www.investopedia.com/terms-beginning-with-i-4769359
https://www.investopedia.com/terms-beginning-with-j-4769360
https://www.investopedia.com/terms-beginning-with-k-4769361
https://www.investopedia.com/terms-beginning-with-l-4769362
https://www.investopedia.com/terms-beginning-with-m-4769363
https://www.investopedia.com/terms-beginning-with-n-4769364
https://www.investopedia.com/terms-beginning-with-o-4769365
https://

In [24]:
sublist_urls = []

for url in urls:
    resp = requests.get(url)
    if resp.status_code == 200:
        page_soup = BeautifulSoup(resp.text, 'html.parser')
        sublist_url_elements = page_soup.find_all('a', class_='dictionary-top300-list__list mntl-text-link')
        sublist_urls.extend([el['href'] for el in sublist_url_elements])

In [25]:
all_data = []

def fetch_and_process(sublist_url):
    retries = 3
    for attempt in range(retries):
        try:
            sublist_response = requests.get(sublist_url, headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            })
            if sublist_response.status_code == 200:
                page_soup = BeautifulSoup(sublist_response.text, 'html.parser')
                div = page_soup.find('div', id='mntl-sc-block-callout-body_1-0')
                if div:
                    list_items = div.find_all('li')
                    definitions = [li.get_text(strip=True) for li in list_items]
                    header = page_soup.find('h1', class_='comp article-heading mntl-text-block')
                    if header:
                        term = header.text.strip()
                        return term, definitions, sublist_url
            else:
                print(f"Failed to fetch {sublist_url}, status code: {sublist_response.status_code}")
        except Exception as e:
            print(f"Error processing {sublist_url}: {e}")
            time.sleep(2)
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    future_to_url = {executor.submit(fetch_and_process, url): url for url in sublist_urls}

    for future in concurrent.futures.as_completed(future_to_url):
        result = future.result()
        if result:
            term, definitions, sublist_url = result
            all_data.append([term, '/n'.join(definitions), sublist_url])


In [26]:
def save_to_csv(filepath, data):
    with open(filepath, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Term', 'Definition', 'URL'])
        writer.writerows(data)


save_to_csv('investopediaScrapper.csv', all_data)

print("Data saved successfully.")

Data saved successfully.


In [27]:
len(all_data)

6064